In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
from datetime import datetime
import ast
import utils.helpers as hpr

### Functions

In [153]:
def time_diff(start, end):
    if start > end:
        start, end = end, start
    current_date =  datetime.strptime(end[:-11], "%Y-%m-%d %H:%M:%S") 
    previous_date = datetime.strptime(start[:-11], "%Y-%m-%d %H:%M:%S") 
    diff = current_date - previous_date
    diff = float("{:.2f}".format(diff.total_seconds() / 3600))
    return diff

def compute_time_diff(revisions):
    if len(revisions) == 1:
        return 0
    dates = [rev["created"] for rev in revisions]
    dates.sort()
    return time_diff(dates[0], dates[-1])

### Main dataframe

In [2]:
def combine_openstack_data():
    '''Combine generated csv files into a single DataFrame object
    '''
    df = pd.DataFrame([])
    data_path = "%sChanges/" % hpr.DIR
    changes_file_names = hpr.list_file(data_path)
    for f in changes_file_names:
        df_per_file = pd.read_csv("%s%s" % (data_path, f))
        df = pd.concat((df, df_per_file))

    df = df.drop_duplicates(subset=["number"])

    df = df.sort_values(by="updated", ascending=False).reset_index(drop=True)

    return df

In [3]:
df = combine_openstack_data()

In [ ]:
df["reviewers"] = df["reviewers"].apply(ast.literal_eval)

In [9]:
df["reviewers_ids"] = df["reviewers"].map(lambda revs: [rev["_account_id"] for rev in revs if "tags" not in rev.keys()])

In [160]:
df["revisions"] = df["revisions"].apply(ast.literal_eval)

In [161]:
df["merge_duration"] = df["revisions"].map(compute_time_diff)

In [3]:
df_all_dependencies = pd.read_csv("./Files/all_dependencies.csv")

In [167]:
len(df_all_dependencies.loc[(df_all_dependencies.is_cross_service==True)
& (df_all_dependencies.Source_status == "MERGED") &
(df_all_dependencies.Target_status == "MERGED") &
(df_all_dependencies.is_source_bot == False) &
(df_all_dependencies.is_target_bot == False) &
(df_all_dependencies.is_same_dev != "Same")])

4182

In [174]:
len(df_all_dependencies[(df_all_dependencies.is_cross)
                    & (df_all_dependencies.Source_status == "MERGED") &
                    (df_all_dependencies.Target_status == "MERGED") &
                    (df_all_dependencies.is_source_bot == False) &
                    (df_all_dependencies.is_target_bot == False) &
                    (df_all_dependencies.is_same_dev == "Different") &
                    ((df_all_dependencies.Source_rev_target == True) &
                     (df_all_dependencies.Target_rev_source == True))
                     ])/len(df_all_dependencies[(df_all_dependencies.is_cross)& 
                                                (df_all_dependencies.Source_status == "MERGED") &
                                                (df_all_dependencies.Target_status == "MERGED") &
                                                (df_all_dependencies.is_source_bot == False) &
                                                (df_all_dependencies.is_target_bot == False) &
                                                (df_all_dependencies.is_same_dev == "Different")])


0.2744555824403733

### The co-changes reviewed by how many shared reviewers ?

In [337]:
df_all_dependencies.loc[(df_all_dependencies["status_source"]=="MERGED")&(df_all_dependencies["status_target"]=="MERGED")&(df_all_dependencies["is_cross"]==True), ["intersect_rev"]].to_csv("./RQs/RQ4/Files/cross_project_changes_reviewers_intersect.csv",index=False)

In [338]:
df_all_dependencies.loc[(df_all_dependencies["status_source"]=="MERGED")&(df_all_dependencies["status_target"]=="MERGED")&(df_all_dependencies["is_cross_service"]==True), ["intersect_rev"]].to_csv("./RQs/RQ4/Files/cross_service_changes_reviewers_intersect.csv",index=False)

In [78]:
df_all_dependencies.loc[
    # (df_all_dependencies["intersect_rev"]) &
                            (df_all_dependencies["is_source_bot"] == False) &
                            (df_all_dependencies["is_target_bot"] == False) &
                            (df_all_dependencies["Source_status"] == "MERGED") &
                            (df_all_dependencies["Target_status"] == "MERGED") &
                            (df_all_dependencies["is_cross_service"] == True),
                            "intersect_rev"].median()

20.0

In [74]:
len(df_all_dependencies.loc[(df_all_dependencies["intersect_rev"] == 0) &
                            (df_all_dependencies["is_source_bot"] == False) &
                            (df_all_dependencies["is_target_bot"] == False) &
                            (df_all_dependencies["Source_status"] == "MERGED") &
                            (df_all_dependencies["Target_status"] == "MERGED") &
                            (df_all_dependencies["is_cross"] == True),
                            "intersect_rev"]) / len(df_all_dependencies[
                                (df_all_dependencies["is_source_bot"] == False) &
                            (df_all_dependencies["is_target_bot"] == False) &
                            (df_all_dependencies["Source_status"] == "MERGED") &
                            (df_all_dependencies["Target_status"] == "MERGED") &
                            (df_all_dependencies["is_cross"] == True)])


0.08659395688338142

In [16]:
df_all_dependencies.loc[(df_all_dependencies["intersect_rev"] > 0) &
                            (df_all_dependencies["is_source_bot"] == False) &
                            (df_all_dependencies["is_target_bot"] == False) &
                            (df_all_dependencies["status_source"] == "MERGED") &
                            (df_all_dependencies["status_target"] == "MERGED") &
                            (df_all_dependencies["is_cross_service"] == True),
                            "intersect_rev"].median()

22.22222222222222

In [64]:
df_all_dependencies.loc[(df_all_dependencies["intersect_rev"] == 0) &
                            (df_all_dependencies["is_source_bot"] == False) &
                            (df_all_dependencies["is_target_bot"] == False) &
                            (df_all_dependencies["status_source"] == "MERGED") &
                            (df_all_dependencies["status_target"] == "MERGED") &
                            (df_all_dependencies["is_cross"] == True)]

,Source,Target,Source_repo,Target_repo,is_cross,Service_source,Service_target,status_source,status_target,is_cross_service,...,min_exp_revision,max_exp_revision,max_min_exp_revision,revision_weak_exp,revision_strong_exp,Source_datetime,Target_datetime,time_diff,max_min_time,time_weak_exp
106,863158,863161,openstack/project-config,openstack/governance,True,NaN,technicalcommittee,MERGED,MERGED,False,...,1.0,3.0,3.0,1.0,3.0,141.85,0.0,0.27,0.0,0.0
156,740601,863783,openstack/puppet-openstack_spec_helper,openstack/puppet-openstack-cookiecutter,True,puppetopenstack,puppetopenstack,MERGED,MERGED,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,863584,863585,openstack/charm-ops-sunbeam,openstack/charm-nova-k8s,True,openstackcharms,nova,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.15,0.0,0.0
182,863076,863077,openstack/governance,openstack/requirements,True,technicalcommittee,requirements,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.05,0.0,0.0
183,861886,863076,openstack/openstack-ansible,openstack/governance,True,openstackansible,technicalcommittee,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,290.23,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187833,826986,826989,openstack/octavia-dashboard,openstack/ironic-ui,True,octavia,ironic,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.00,0.0,0.0
187834,826987,826988,openstack/monasca-ui,openstack/magnum-ui,True,monasca,magnum,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.00,0.0,0.0
187835,826987,826989,openstack/monasca-ui,openstack/ironic-ui,True,monasca,ironic,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.00,0.0,0.0
187836,826988,826989,openstack/magnum-ui,openstack/ironic-ui,True,magnum,ironic,MERGED,MERGED,True,...,1.0,1.0,1.0,1.0,1.0,0.00,0.0,0.00,0.0,0.0


In [112]:
df_all_dependencies.to_csv("./Files/all_dependencies.csv", index=False)

### How much experience a developer has cross-project changes ? 

In [33]:
len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["min"] == 0) &
    (df_all_dependencies["is_same_dev"] == "Same")])/len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same")])

0.20179435257059286

In [180]:
len(df_all_dependencies[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["is_same_dev"] == "Different") &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    ((df_all_dependencies["is_source_comment"] == True)| 
        (df_all_dependencies["is_target_comment"] == True))])/len(df_all_dependencies[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["is_same_dev"] == "Different") &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    ((df_all_dependencies["Source_rev_target"] == True)| 
        (df_all_dependencies["Target_rev_source"] == True))])

0.2112183353437877

In [ ]:
df_all_dependencies_summarized = pd.read_csv("./Files/all_dependencies_summarized.csv")

In [ ]:
df_all_dependencies_summarized[
    (df_all_dependencies_summarized["Duration"] >= 125) &
    (df_all_dependencies_summarized["Project_type"] == "Target")
    & (df_all_dependencies_summarized["is_same_dev"] == "Same")].sort_values("Duration")

#### 2nd finding

##### Churn

In [58]:
df_all_dependencies_churn_less = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same"), ["Churn_less"]]
df_all_dependencies_churn_less["Label"] = "Less familiar"
df_all_dependencies_churn_less.rename(columns={"Churn_less": "Churn"}, inplace=True)

df_all_dependencies_churn_more = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same"), ["Churn_more"]]
df_all_dependencies_churn_more["Label"] = "More familiar"
df_all_dependencies_churn_more.rename(columns={"Churn_more": "Churn"}, inplace=True)

df_all_dependencies_churn = pd.concat((df_all_dependencies_churn_less, df_all_dependencies_churn_more))
df_all_dependencies_churn[df_all_dependencies_churn.Churn.notnull()].to_csv("./RQs/RQ4/Files/Same_dev_churn.csv", index=False)

In [53]:
df_all_dependencies_churn_more.median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1753/1272737896.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_all_dependencies_churn_more.median()


Churn    11.0
dtype: float64

#### Duration

In [97]:
df_all_dependencies_duration_less = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same"), ["Duration_less"]]
df_all_dependencies_duration_less["Label"] = "Less familiar"
df_all_dependencies_duration_less.rename(columns={"Duration_less": "Duration"}, inplace=True)

df_all_dependencies_duration_more = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same"), ["Duration_more"]]
df_all_dependencies_duration_more["Label"] = "More familiar"
df_all_dependencies_duration_more.rename(columns={"Duration_more": "Duration"}, inplace=True)
df_all_dependencies_duration = pd.concat((df_all_dependencies_duration_less, df_all_dependencies_duration_more))
df_all_dependencies_duration[df_all_dependencies_duration.Duration.notnull()].to_csv("./RQs/RQ4/Files/Same_dev_duration.csv", index=False)

In [101]:
df_all_dependencies_duration[df_all_dependencies_duration.Label=="More familiar"].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1753/238608310.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_all_dependencies_duration[df_all_dependencies_duration.Label=="More familiar"].median()


Duration    106.42
dtype: float64

#### Number of revisions

In [60]:
df_all_dependencies_revisions_less = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same") &
    (df_all_dependencies["Revisions_less"] > 0), ["Revisions_less"]]
df_all_dependencies_revisions_less["Label"] = "Less familiar"
df_all_dependencies_revisions_less.rename(columns={"Revisions_less": "Revisions"}, inplace=True)

df_all_dependencies_revisions_more = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["Source_status"] == "MERGED") &
    (df_all_dependencies["Target_status"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    (df_all_dependencies["is_same_dev"] == "Same") &
    (df_all_dependencies["Revisions_less"] > 0), ["Revisions_more"]]
df_all_dependencies_revisions_more["Label"] = "More familiar"
df_all_dependencies_revisions_more.rename(columns={"Revisions_more": "Revisions"}, inplace=True)
df_all_dependencies_revisions = pd.concat((df_all_dependencies_revisions_less, df_all_dependencies_revisions_more))
df_all_dependencies_revisions[df_all_dependencies_revisions.Revisions.notnull()].to_csv("./RQs/RQ4/Files/Same_dev_revisions.csv", index=False)

In [71]:
df_all_dependencies_revisions[(df_all_dependencies_revisions.Revisions.notnull())&(df_all_dependencies_revisions.Label=="More familiar")].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1753/1670072075.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_all_dependencies_revisions[(df_all_dependencies_revisions.Revisions.notnull())&(df_all_dependencies_revisions.Label=="More familiar")].median()


Revisions    2.0
dtype: float64

#### Does it take much time to merge changes in projects with less experience vs. changes in projects with familiar with

In [ ]:
df_all_dependencies["Source_duration"] = df_all_dependencies.loc[(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True), "Source"].map(lambda nbr: df.loc[df.number==nbr,"merge_duration"].values[0])
df_all_dependencies["Target_duration"] = df_all_dependencies.loc[(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True), "Target"].map(lambda nbr: df.loc[df.number==nbr,"merge_duration"].values[0])
df_all_dependencies["merge_duration_diff"] = df_all_dependencies.loc[(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True),["Source_exp", "Target_exp", "Source_duration", "Target_duration"]].apply(identify_merge_duration, axis=1)

In [211]:
len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["status_source"] == "MERGED") &
    (df_all_dependencies["status_target"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    ((df_all_dependencies["Source_exp"] != 0) |
     (df_all_dependencies["Target_exp"] != 0)) &
    (df_all_dependencies["same_dev"] == True) &
    (df_all_dependencies["merge_duration_diff"] > 1), ["min"]]) / len(
        df_all_dependencies.loc[
            (df_all_dependencies["is_cross"] == True) &
            ((df_all_dependencies["Source_exp"] != 0) & 
            (df_all_dependencies["Target_exp"] != 0)) &
            (df_all_dependencies["status_source"] == "MERGED") &
            (df_all_dependencies["status_target"] == "MERGED") &
            (df_all_dependencies["is_source_bot"] == False) &
            (df_all_dependencies["is_target_bot"] == False) &
            (df_all_dependencies["same_dev"] == True)])


0.4403501703234314

In [ ]:
df_all_dependencies.loc[
    (df_all_dependencies["is_cross"] == True) &
    (df_all_dependencies["status_source"] == "MERGED") &
    (df_all_dependencies["status_target"] == "MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    ((df_all_dependencies["Source_exp"] != 0) |
     (df_all_dependencies["Target_exp"] != 0)) &
    (df_all_dependencies["same_dev"] == True) &
    (df_all_dependencies["merge_duration_diff"] < 1), ["min"]]

In [136]:
df_all_dependencies.loc[(df_all_dependencies["status_source"]=="MERGED")&(df_all_dependencies["status_target"]=="MERGED")&(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True), ["min", "max", "max_min"]].to_csv("./RQs/RQ4/Files/same_dev_experience.csv", index=False)

### Analysis

In [ ]:
df_all_dependencies["Source_revisions"] = df_all_dependencies.loc[(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True), "Source"].map(lambda nbr: df.loc[df["number"]==nbr,"revisions_count"].values[0])
df_all_dependencies["Target_revisions"] = df_all_dependencies.loc[(df_all_dependencies["is_cross"]==True)&(df_all_dependencies["same_dev"]==True), "Target"].map(lambda nbr: df.loc[df["number"]==nbr,"revisions_count"].values[0])

In [8]:
df_all_dependencies = pd.read_csv("./Files/all_dependencies.csv")

In [215]:
len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["intersect_rev"]==0) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False)])/len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["status_source"]=="MERGED") &
    (df_all_dependencies["status_target"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False)])

0.08718834644997547

In [79]:
len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["intersect_rev"]==0) &
    (df_all_dependencies["Source_release"]!=df_all_dependencies["Target_release"]) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False), ["Source_release", "Target_release"]])/len(df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["intersect_rev"]==0) &
    # (df_all_dependencies["Source_release"]!=df_all_dependencies["Target_release"]) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False), ["Source_release", "Target_release"]])

0.5094339622641509

#### Does it get quickly reviewed ?

In [305]:
def compute_review_duration(nbr):
    messages = df.loc[df.number==nbr, "messages"].values[0]
    messages.sort(key=lambda x: x["date"])
    if len(messages) >1:
        return time_diff(messages[0]["date"], messages[-1]["date"])
    return 0

In [5]:
df_all_dependencies_duration_source_reviewers = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    # (df_all_dependencies["Duration_source"].notnull()) &
    (df_all_dependencies["intersect_rev"]>0), ["Source_duration"]]
df_all_dependencies_duration_source_reviewers.rename(columns={"Source_duration": "Duration"}, inplace=True)
df_all_dependencies_duration_source_reviewers["Reviewer_sharing"] = "At least one"
df_all_dependencies_duration_source_reviewers["Project_type"] = "Source"

df_all_dependencies_duration_target_reviewers = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    # (df_all_dependencies["Duration_target"].notnull())&
    (df_all_dependencies["intersect_rev"]>0), ["Target_duration"]]
df_all_dependencies_duration_target_reviewers.rename(columns={"Target_duration": "Duration"}, inplace=True)
df_all_dependencies_duration_target_reviewers["Reviewer_sharing"] = "At least one"
df_all_dependencies_duration_target_reviewers["Project_type"] = "Target"

df_all_dependencies_duration_source_no_reviewers = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    # (df_all_dependencies["Duration_source"].notnull()) &
    (df_all_dependencies["intersect_rev"]==0), ["Source_duration"]]
df_all_dependencies_duration_source_no_reviewers.rename(columns={"Source_duration": "Duration"}, inplace=True)
df_all_dependencies_duration_source_no_reviewers["Reviewer_sharing"] = "Different"
df_all_dependencies_duration_source_no_reviewers["Project_type"] = "Source"

df_all_dependencies_duration_target_no_reviewers = df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    # (df_all_dependencies["Duration_target"].notnull()) &
    (df_all_dependencies["intersect_rev"]==0), ["Target_duration"]]
df_all_dependencies_duration_target_no_reviewers.rename(columns={"Target_duration": "Duration"}, inplace=True)
df_all_dependencies_duration_target_no_reviewers["Reviewer_sharing"] = "Different"
df_all_dependencies_duration_target_no_reviewers["Project_type"] = "Target"


In [6]:
df_all_dependencies_duration_source_no_reviewers.median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_2190/1359031094.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_all_dependencies_duration_source_no_reviewers.median()


Duration    70.81
dtype: float64

In [7]:
df_all_dependencies_duration_reviewers = pd.concat((df_all_dependencies_duration_source_reviewers,df_all_dependencies_duration_target_reviewers,df_all_dependencies_duration_source_no_reviewers,df_all_dependencies_duration_target_no_reviewers))

In [130]:
df_all_dependencies.loc[
    (df_all_dependencies["is_cross"]==True) &
    (df_all_dependencies["Source_status"]=="MERGED") &
    (df_all_dependencies["Target_status"]=="MERGED") &
    (df_all_dependencies["is_source_bot"] == False) &
    (df_all_dependencies["is_target_bot"] == False) &
    # (df_all_dependencies["Duration_more"].notnull())&
    (df_all_dependencies["intersect_rev"]==0), ["Duration_less"]].median()

Duration_less    71.72
dtype: float64

In [140]:
df_all_dependencies_duration_reviewers[
    (df_all_dependencies_duration_reviewers["Reviewer_sharing"]=="(+) reviewer") &
    (df_all_dependencies_duration_reviewers["Label"]=="Max duration")].median()

/var/folders/p4/wn9d12jj6g78td0tl7gx7xlw0000gp/T/ipykernel_1753/768293591.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_all_dependencies_duration_reviewers[


Source_duration     92.81
Target_duration    123.83
Duration           202.09
dtype: float64

In [8]:
df_all_dependencies_duration_reviewers.to_csv("./RQs/RQ4/Files/Duration_reviewers.csv", index=False)

In [18]:
df_all_dependencies.to_csv("./Files/all_dependencies.csv", index=False)